The purpose of this script is to retrive the following information from the OpenET API:  
-all 6 models and the Ensemble estimates of ET  
-dates of landsat overpass  
-ET, ETo, ETof, and Precipitation values extracted from SIMS/CIMIS

This script carries out the above operations for both the lettuce and broccoli data using dynamic footprints

Input files:  
Dynamic footprint JSON

Output files:  
-.csv holding all models and Ensemble estimates of ET  
-.csv holding dates of landsat overpass  
-.csv holding SIMS ET as well as ETo, ETof, and Precip values collected via CIMIS

###Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/home')

Mounted at /content/home


###Import Dependencies

In [2]:
import requests
import json
import pandas as pd
import geopandas as gpd
import numpy as np
#from matplotlib.dates import DateFormatter
#import matplotlib.pyplot as plt

###Declare Custom Functions

In [3]:
def openET_API_polygon_query_ET (MY_API_KEY, START_DATE, END_DATE, INTERVAL, GEOMETRY, UNIT):
  '''
  MY_API_KEY: string
  START_DATE: inclusive, string
  END_DATE: inclusive, string
  INTERVAL: 'daily' or 'monthly'
  GEOMETRY: array of coordinates
  UNIT: 'mm' or 'in'

  '''
  #string of all available models
  models = ['Ensemble','geeSEBAL','SSEBop','SIMS','DisALEXI','PTJPL','eeMETRIC']

  #create df to load up
  API_df = pd.DataFrame()

  for model in models:
    # set your API key before making the request
    header = {"Authorization": MY_API_KEY}

    # endpoint arguments
    args = {
      "date_range": [
        START_DATE,
        END_DATE
      ],
      "interval": INTERVAL,
      "geometry": GEOMETRY,
      "model": model,
      "variable": 'ET',
      "reference_et": "CIMIS",
      "reducer": "mean",
      "units": UNIT,
      "file_format": "JSON"
    }

    # query the api
    resp = requests.post(
        headers=header,
        json=args,
        url="https://openet-api.org/raster/timeseries/polygon"
    )

    #print(resp.text)
    intermediate_API_df = pd.read_json(resp.text)
    API_df['date'] = intermediate_API_df['time']
    API_df[model] = intermediate_API_df['et']

    #print some statements to show what step
    print(model, end='')
    print(" done!")

  #return df
  print("dataframe created :)")
  return API_df

###Set Directories and Declare Variables

In [4]:
MAIN_DIR = '/content/home/MyDrive/B_SanLuis'

#'TRUE' or 'FALSE' depending on whether you would like to export the dataframes as .csv
SAVE_EXPORTS = 'FALSE'

#Choose between 'Broccoli2023' or 'Broccoli2024'
CROP_OF_CONCERN = 'Broccoli2023'

#RS OpenET API key. Please don't share outside of this doc
MY_KEY = '095Ip2n5f3pJFLh4YHdk6CSdbPezttwadaEgJ5ZHZSSbKXszgLmP51nKIt7i'

if CROP_OF_CONCERN == 'Broccoli2023':
  START_DATE = '2023-07-21'
  END_DATE = '2023-10-04'
  OVERPASS_START_DATE = '2023-07-13'  # +/-8 days from start/end
  OVERPASS_END_DATE = '2023-10-12'
  GEOMETRY_FILE = MAIN_DIR + '/Data/Final_Json/Broccoli2023Final.geojson'
  OUT_FILE =  MAIN_DIR + '/Data/OpenET/Broccoli2023_OpenET.csv'
  print("Geometry sourced from: ")
  print(GEOMETRY_FILE)
  print("Files will be saved at: ")
  print(OUT_FILE)

if CROP_OF_CONCERN == 'Broccoli2024':
  START_DATE = '2024-07-25'
  END_DATE = '2024-10-18'
  OVERPASS_START_DATE = '2024-07-18'  # +/-8 days from start/end
  OVERPASS_END_DATE = '2024-10-26'
  GEOMETRY_FILE = MAIN_DIR + '/Data/Final_Json/Broccoli2023Final.geojson'
  OUT_FILE =  MAIN_DIR + '/Data/Intermediate_Datasets/Broccoli/Broccoli2024_OpenET.csv'
  print("Geometry sourced from: ")
  print(GEOMETRY_FILE)
  print("Files will be saved at: ")
  print(OUT_FILE)

Geometry sourced from: 
/content/home/MyDrive/B_SanLuis/Data/Final_Json/Broccoli2023Final.geojson
Files will be saved at: 
/content/home/MyDrive/B_SanLuis/Data/OpenET/Broccoli2023_OpenET.csv


###Create Coordinate List for OpenET

In [5]:
FOOTPRINT_GEOM = gpd.read_file(GEOMETRY_FILE)
FOOTPRINT_GEOM_JSON = json.loads(FOOTPRINT_GEOM.to_json())

In [6]:
MY_COORDS = np.array(FOOTPRINT_GEOM_JSON["features"][0]["geometry"]["coordinates"])

In [7]:
COORD_LIST = ([])
for i in range(0,MY_COORDS.shape[1]):
  COORD_LIST.append(MY_COORDS[0][i][0])
  COORD_LIST.append(MY_COORDS[0][i][1])

###Run API Queries

In [8]:
ET_df = openET_API_polygon_query_ET(MY_KEY,START_DATE,END_DATE,'daily',COORD_LIST,'mm')

<ipython-input-3-5904a3c7ec4d>:45: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  intermediate_API_df = pd.read_json(resp.text)


ValueError: If using all scalar values, you must pass an index

In [ ]:
ET_df

###Export .csv files

In [ ]:
if SAVE_EXPORTS == 'TRUE':
  #export daily data as .csv
  #ET_df.to_csv(OUT_FILE)
  #overpass_df.to_csv(OVERPASS_OUT_FILE)
  SIMS_all_df.to_csv(SIMS_OUT_FILE)
  print('files saved at:')
  print(OUT_FILE)